In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm

In [ ]:
from epimodel.pymc3_models.epi_params import bootstrapped_negbinom_values, ci_to_mean_sd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

# infected to reporting

In [ ]:
symptom_reporting = {
    'mean_mean': 5.82,
    'mean_sd': 0.68,
    'disp_mean': 1.57,
    'disp_sd': 0.054,
    'source': 'Cereda et al, https://arxiv.org/ftp/arxiv/papers/2003/2003.09320.pdf, §3'
              'Fonfria et al, https://www.medrxiv.org/content/medrxiv/early/2020/06/19/2020.06.17.20133587',
    'dist': 'negbinom',
    'notes': 'mean from Fronfria et al. Dispersions from Cereda et al.'
}

symptom_deaths = {
    'mean_mean': 16.71,
    'mean_sd': 0.75,
    'sd_mean': 6.9,
    'sd_sd': 1.122,
    'source': 'Linton et al'
              'Fonfria et al, https://www.medrxiv.org/content/medrxiv/early/2020/06/19/2020.06.17.20133587',
    'dist': 'gamma',
    'notes': 'Mean from Fronfria et al. Sd from Linton et al'
}

In [ ]:
import scipy.stats as stats

In [ ]:
w = stats.weibull_min(c=2.82)

In [ ]:
5.665 * w.mean()

In [ ]:
ci_to_mean_sd(2.826, np.array([1.75, 4.7]))

In [ ]:
ci_to_mean_sd(5.665, np.array([4.7, 6.9]))

In [ ]:
5.665 * w.std()

In [ ]:
nRVs = 2500

ms = np.zeros(nRVs)
sds = np.zeros(nRVs)

for i in tqdm(range(nRVs)):
    shape = np.random.uniform(1.75, 4.7)
    scale = np.random.uniform(4.7, 6.9)
#     shape=  max(np.random.normal(loc=2.926, scale=0.96), 1.5)
#     scale = max(np.random.normal(loc=5.665, scale=0.63), 3)
    w = stats.weibull_min(c=shape)
    ms[i] = scale * w.mean()
    sds[i] = scale * w.std()

print(np.mean(ms))
print(np.std(ms))   
print(np.mean(sds))
print(np.std(sds))

In [ ]:
ci_data = np.loadtxt('ci_data.csv', delimiter=',', skiprows=1, usecols=[1, 2, 4])

In [ ]:
means = np.exp(ci_data[:, 0]) / np.exp(ci_data[:, 1])
sigmas = np.exp(0.5*ci_data[:, 0]) / np.exp(ci_data[:, 1])

nLogL = ci_data[:, 2]

In [ ]:
nLogL < 3.84

In [ ]:
np.min(sigmas[nLogL < 3.84])

In [ ]:
np.max(sigmas[nLogL < 3.84])

In [ ]:
sd_middle = 2.11

In [ ]:
ci_to_mean_sd(2.11, np.array([1.525, 3.09]))

In [ ]:
np.min(means[nLogL < 3.84])

In [ ]:
np.max(means[nLogL < 3.84])

In [ ]:
plt.plot(ms)

In [ ]:
ms = np.linspace(1.5, 1.55, 51)
sds = np.linspace(0.05, 0.1, 51)

nRVs = int(1e7)

noise_mean = np.random.normal(0, 1, size=nRVs)
noise_sd = np.random.normal(0, 1, size=nRVs)

min_sd = 100
best_sol = np.array([0., 0.])
tol = 0.01

sd_mean = 0.418
sd_sd = 0.0759

for mean_mean in tqdm(ms.tolist()):
    for mean_sd in sds:
        log_mean = mean_mean + noise_mean * mean_sd
        log_sd = sd_mean + noise_sd * sd_sd
        means = np.exp(log_mean + 0.5 * log_sd ** 2)
        m = np.mean(means)
        l = np.percentile(means, 2.5)
        u = np.percentile(means, 97.5)
        if np.abs(m - 5.06) < tol:
            if l < 5.06 and u > 5.7:
                if mean_sd < min_sd:
                    min_sd = mean_sd
                    best_sol[0] = mean_mean
                    best_sol[1] = mean_sd
print(f'Best solution: {best_sol}')

In [ ]:
# final 
incubation_period = {
                'mean_mean': 1.53,
                'mean_sd': 0.051,
                'sd_mean': 0.418,
                'sd_sd': 0.0759,
                'source': 'Lauer et al, doi.org/10.7326/M20-0504'
                          'Fonfria et al, https://www.medrxiv.org/content/medrxiv/early/2020/06/19/2020.06.17.20133587',
                'dist': 'lognorm',
                'notes': 'mean_mean, mean_sd chosen from Fonfria et al, after fitting using Lauer values.'
                         '(log) sd, sd_sd taken from Lauer et al.'
            }

In [ ]:
bootstrapped_negbinom_values([incubation_period, symptom_reporting])

In [ ]:
p, ms, disps = bootstrapped_negbinom_values([incubation_period, symptom_deaths], truncation=64)

In [ ]:
p